# Data Engineering

### 왜 데이터 모판을 만드는가
- train data 와 test data에 같은 전처리 과정을 해줘야함
- 데이터에 따라 train에만 있고 Test에는 존재하지 않는 column이 있을 수 있음
  - 초기 데이터 전처리 시 규칙을 만들어야함
- 데이터의 분포를 좀 더 넓게 볼 수 있음

In [2]:
import pandas as pd
import os

In [83]:
test_file_path = os.path.join('..','data','titanic', 'test.csv')
train_file_path = os.path.join('..','data','titanic','train.csv')
test_df = pd.read_csv(test_file_path)
train_df = pd.read_csv(train_file_path)

df = pd.concat([train_df, test_df])
df = df.reset_index(drop=True)

number_of_train = df.Survived.notnull().sum()

y = df.pop('Survived')[:number_of_train]
y

0      0.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
886    0.0
887    1.0
888    0.0
889    1.0
890    0.0
Name: Survived, Length: 891, dtype: float64

## 데이터 처리 전략

- 하나의 셀은 다시 실행해도 그 결과가 보장돼야 한다 (특히 jupyter notebook)
- 전처리 완료 후 함수화
- 컬럼이름은 List로 관리
- 데이터는 타입별로 분리하여 보관
- 데이터 노트 작성
  - 데이터에 대한 처리 내용 및 방향을 정리한 노트
  - 기본적인 전처리 방향과 방법을 정리한 것
  - 데이터에 대한 아이디어를 정리와 지속적인 업데이트

### 기본적인 데이터 현황 파악 코드

```python
df.dtypes
df.info()
df.isnull.sum()
df.describe()
df.head(2).T
```

### Data Cleansing issues
- 데이터가 빠진 경우 (결측치의 처리)
- 라벨링된 데이터의 처리(category)
- 데이터의 Scale의 차이가 크게 날 경우

## Missing Value Strategy
- 데이터가 없으면 row를 drop
  - 데이터가 없는 최소 개수를 정해서 row drop
- 데이터가 거의 없는 feature는 아예 column drop
- 최빈값, 평균값 등으로 비어있는 데이터를 채움

### data drop

- `df.dropna()` : NaN 값이 있는 row를 삭제
- `df.dropna(how = 'all')` : 모든 column이 NaN 값이면 row를 삭제
- `df.dropna(axis = 1)` : NaN 값이 있는 column 삭제
- `df.dropna(axis = 1, thresh = 3)` : NaN 값이 최소 4개 이상 있는 column 삭제

### fill data

- 평균값, 최빈값, 중앙값 등으로 값을 채우는게 일반적

- 특정 조건에 따라 다른 값을 채우고 싶을 때 `transform` 이용

In [67]:
import numpy as np

In [78]:
raw_data = {'first_name': ['Jason', np.nan, 'Tina', 'Jake', 'Amy'], 
        'last_name': ['Miller', np.nan, 'Ali', 'Milner', 'Cooze'], 
        'age': [42, np.nan, 36, 24, 73], 
        'sex': ['m', np.nan, 'f', 'm', 'f'], 
        'preTestScore': [4, np.nan, np.nan, 2, 3],
        'postTestScore': [25, np.nan, np.nan, 62, 70]}
ex_df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'sex', 'preTestScore', 'postTestScore'])
ex_df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [79]:
# 모든 데이터가 NaN인거 drop
cleaned_df = ex_df.dropna(axis=1, how="all")
cleaned_df = cleaned_df.dropna(axis=0, how="all")
cleaned_df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [80]:
fillna_df = cleaned_df.copy()
fillna_df["preTestScore"] = cleaned_df["preTestScore"].fillna(cleaned_df["preTestScore"].mean())
fillna_df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
2,Tina,Ali,36.0,f,3.0,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [81]:
# 성별로 나눈 후 평균값을 할당
cleaned_df.groupby("sex")["postTestScore"].transform("mean")

0    43.5
2    70.0
3    43.5
4    70.0
Name: postTestScore, dtype: float64

In [77]:
# NaN값에 transformg한게 들어간다
fillna_df["postTestScore"] = cleaned_df["postTestScore"].fillna(
    cleaned_df.groupby("sex")["postTestScore"].transform("mean")
)

fillna_df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
2,Tina,Ali,36.0,f,3.0,70.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


## titanic example

In [84]:
df[df["Age"].notnull()].groupby(["Sex"])["Age"].mean()

Sex
female    28.687088
male      30.585228
Name: Age, dtype: float64

In [94]:
df.groupby("Pclass")["Age"].transform("mean")

0       24.816367
1       39.159930
2       24.816367
3       39.159930
4       24.816367
          ...    
1304    24.816367
1305    39.159930
1306    24.816367
1307    24.816367
1308    24.816367
Name: Age, Length: 1309, dtype: float64

In [95]:
df["Age"].fillna(
    df.groupby("Pclass")["Age"].transform("mean"), inplace =True)
df.isnull().sum() / len(df) * 100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age             0.000000
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.076394
Cabin          77.463713
Embarked        0.152788
dtype: float64